In [1]:
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import string
import pandas as pd
import cv2

def mymodel():
    model = Sequential()

    model.add(Conv2D(16,(3,3),activation='relu',input_shape=(128,128,1),padding='same'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    
    model.add(Conv2D(32,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(5,5),activation='relu',padding='same')) 
    model.add(BatchNormalization())
    model.add(Conv2D(32,(5,5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32,(5,5),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3,3)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64,(5,5),activation='relu',padding='same')) 
    model.add(BatchNormalization())
    model.add(MaxPooling2D((3,3)))
    model.add(Dropout(0.3))
    
    model.add(Flatten())

    model.add(Dense(128,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))
    model.add(Dense(64,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Dense(1,activation='sigmoid'))
    return model
    
#1. 데이터
x_train = np.load('/content/drive/MyDrive/dacon/npy/x_train_128.npy')
x_test = np.load('/content/drive/MyDrive/dacon/npy/x_test_128.npy')
answer = pd.read_csv('/content/drive/MyDrive/dacon/dirty_mnist_answer.csv')
sub = pd.read_csv('/content/drive/MyDrive/dacon/sample_submission.csv', header = 0)

# print(x_train.shape) (50000, 128, 128, 1)
# print(np.max(x_train[0])) # 1.0

alphabets = string.ascii_lowercase
alphabets = list(alphabets)

val = []
acc = []
for alphabet in alphabets:
    y_train = answer.loc[:,alphabet].to_numpy()

    #2. 모델
    model = mymodel()

    #3. 컴파일 훈련
    es = EarlyStopping(patience = 30)
    lr = ReduceLROnPlateau(patience = 10, factor = 0.25, verbose = 1)
    cp = ModelCheckpoint(f'/content/drive/MyDrive/dacon/mcp/{alphabet}.hdf5')
    model.compile(loss = 'binary_crossentropy', optimizer= 'adam', metrics = ['acc'])
    history = model.fit(x_train, y_train, epochs = 10000, batch_size = 128, validation_split = 0.2, callbacks = [es,cp,lr])
    val.append(history.history['val_loss'][-31])
    acc.append(history.history['val_acc'][-31])

print('loss : ', val)
print('acc : ', acc)
    #4. 평가 예측
#     y_pred = model.predict(x_test)
#     y_pred = np.where(y_pred<0.5,0,1)
#     sub.loc[:,alphabet] = y_pred

# sub.to_csv('/content/drive/MyDrive/dacon/submission_003.csv', index = 0)

# loss :  [0.6772362589836121, 0.5581411719322205, 0.38123103976249695, 0.6617963910102844, 0.37605029344558716, 0.6572653651237488, 0.5549827814102173, 0.5124793648719788, 0.4877137839794159, 0.4501129388809204, 0.3214549124240875, 0.5551599860191345, 0.5238714218139648, 0.683418869972229, 0.3097933530807495, 0.4915141463279724, 0.620019257068634, 0.6751967072486877, 0.1978851854801178, 0.6415449380874634, 0.5357807874679565, 0.6829854249954224, 0.4044191539287567, 0.35578519105911255, 0.6477894186973572, 0.3420696556568146]
# acc :  [0.569599986076355, 0.7466999888420105, 0.8467000126838684, 0.6310999989509583, 0.8379999995231628, 0.6625999808311462, 0.7145000100135803, 0.7595999836921692, 0.7706000208854675, 0.7932999730110168, 0.8715999722480774, 0.7429999709129333, 0.7480000257492065, 0.5593000054359436, 0.8701000213623047, 0.7818999886512756, 0.6916999816894531, 0.5810999870300293, 0.9259999990463257, 0.6399000287055969, 0.7461000084877014, 0.5634999871253967, 0.8270999789237976, 0.8568000197410583, 0.6462000012397766, 0.8622999787330627]

Epoch 1/10000
313/313 [==============================] - 47s 129ms/step - loss: 0.8387 - acc: 0.5149 - val_loss: 1.1307 - val_acc: 0.5388
Epoch 2/10000
313/313 [==============================] - 39s 124ms/step - loss: 0.7034 - acc: 0.5492 - val_loss: 0.7257 - val_acc: 0.5406
Epoch 3/10000
313/313 [==============================] - 39s 124ms/step - loss: 0.6804 - acc: 0.5701 - val_loss: 0.6796 - val_acc: 0.5645
Epoch 4/10000
313/313 [==============================] - 39s 124ms/step - loss: 0.6773 - acc: 0.5724 - val_loss: 0.6957 - val_acc: 0.5431
Epoch 5/10000
313/313 [==============================] - 39s 125ms/step - loss: 0.6710 - acc: 0.5853 - val_loss: 0.6772 - val_acc: 0.5696
Epoch 6/10000
313/313 [==============================] - 39s 124ms/step - loss: 0.6695 - acc: 0.5883 - val_loss: 0.7242 - val_acc: 0.5500
Epoch 7/10000
313/313 [==============================] - 39s 124ms/step - loss: 0.6677 - acc: 0.5896 - val_loss: 0.7465 - val_acc: 0.5431
Epoch 8/10000
313/313 [===========